In [33]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
from tqdm import tqdm 
import csv
import ast 

In [34]:
df= pd.read_csv("train.csv")

In [35]:
print(df.describe())

       untagged_sentence tagged_sentence
count              47340           47340
unique             46610           46637
top                ['.']    [(')', ')')]
freq                  52              46


In [36]:
print(df.head(n=10))

                                   untagged_sentence  \
0  ['The', 'jury', 'further', 'said', 'in', 'term...   
1  ['The', 'September-October', 'term', 'jury', '...   
2  ['``', 'Only', 'a', 'relative', 'handful', 'of...   
3  ['The', 'jury', 'said', 'it', 'did', 'find', '...   
4  ['It', 'recommended', 'that', 'Fulton', 'legis...   
5  ['The', 'grand', 'jury', 'commented', 'on', 'a...   
6                             ['Merger', 'proposed']   
7  ['However', ',', 'the', 'jury', 'said', 'it', ...   
8  ['The', 'City', 'Purchasing', 'Department', ',...   
9  ['It', 'urged', 'that', 'the', 'city', '``', '...   

                                     tagged_sentence  
0  [('The', 'AT'), ('jury', 'NN'), ('further', 'R...  
1  [('The', 'AT'), ('September-October', 'NP'), (...  
2  [('``', '``'), ('Only', 'RB'), ('a', 'AT'), ('...  
3  [('The', 'AT'), ('jury', 'NN'), ('said', 'VB')...  
4  [('It', 'PP'), ('recommended', 'VB'), ('that',...  
5  [('The', 'AT'), ('grand', 'JJ'), ('jury', 'NN'... 

In [37]:
data = []
for index, row in tqdm(df.iterrows()):
    data.append(ast.literal_eval(row['tagged_sentence'])) # changing data-type of entries from 'str' to 'list'

47340it [00:09, 4902.97it/s]


In [38]:

dict_train_tokens={}
dict_train_tags={}
len_of_data= len(data)
for lists in data:
    for tuples in lists:
        token= tuples[0]
        tag= tuples[1]
    
        if token in dict_train_tokens:
            dict_train_tokens[token]+=1
        else:
            dict_train_tokens[token]=1
            
        if tag in dict_train_tags:
            dict_train_tags[tag]+=1
        else:
            dict_train_tags[tag]=1
        
             


In [39]:
len_train_tags=len(dict_train_tags)
len_train_tokens=len(dict_train_tokens)
print(len_train_tags)

49


In [40]:
dict_index_to_tags={}
dict_tags_to_index={}

idx=0

for tag in dict_train_tags:
    if idx not in dict_index_to_tags:
        dict_index_to_tags[idx]= tag 
        dict_tags_to_index[tag]= idx 
    idx+=1
    

In [41]:
 
print(idx)

49


In [42]:
dict_index_to_tokens={}
dict_tokens_to_index={}
idx=0

for token in dict_train_tokens:
    if idx not in dict_index_to_tokens:
        dict_index_to_tokens[idx]= token
        dict_tokens_to_index[token]=idx
    idx+=1

In [43]:
#Emission Matrix
emission_prob_matrix= np.zeros(( len_train_tags,len_train_tokens))
for lists in data:
    for tuples in lists:
        token= tuples[0]
        tag= tuples[1]
        
        emission_prob_matrix[dict_tags_to_index[tag]][dict_tokens_to_index[token]]+=1
        
        
print(emission_prob_matrix.shape)


row_sums = emission_prob_matrix.sum(axis=1)

for i, row_sum in enumerate(row_sums):
    if row_sum > 0:
        emission_prob_matrix[i,:] /= row_sum


(49, 51208)


In [44]:
#Transition Matrix
transition_prob_matrix= np.zeros((len_train_tags+1, len_train_tags+1))

for list in data:
    len_list= len(list)
    for tuple_index in range(len_list-1):
        first_tuple_tag= list[tuple_index][1] 
        second_tuple_tag= list[tuple_index+1][1]
        if tuple_index==0:
            transition_prob_matrix[len_train_tags][dict_tags_to_index[first_tuple_tag]]+=1
            
        elif tuple_index==len_list-1-1:
            transition_prob_matrix[dict_tags_to_index[second_tuple_tag]][len_train_tags]+=1
        
        
        transition_prob_matrix[dict_tags_to_index[first_tuple_tag]][dict_tags_to_index[second_tuple_tag]]+=1
            
            
print(transition_prob_matrix.shape)  
row_sums = transition_prob_matrix.sum(axis=1)
for i, row_sum in enumerate(row_sums):
    if row_sum > 0:
        transition_prob_matrix[i, :] /= row_sum
print(transition_prob_matrix)
            

            
    


(50, 50)
[[1.22478474e-05 6.08901736e-01 4.02954181e-03 ... 0.00000000e+00
  1.22478474e-05 1.22478474e-04]
 [6.72132634e-03 1.03481754e-01 1.92144583e-02 ... 0.00000000e+00
  5.86782458e-05 2.22443896e-03]
 [5.54844490e-02 1.57749904e-02 2.81901958e-02 ... 0.00000000e+00
  0.00000000e+00 3.83975426e-04]
 ...
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 1.11111111e-01]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  8.23129252e-01 6.80272109e-03]
 [1.47079447e-01 5.88360283e-02 6.72715297e-02 ... 0.00000000e+00
  6.37443427e-05 0.00000000e+00]]


In [45]:
print(emission_prob_matrix.shape)
print(transition_prob_matrix.shape)

(49, 51208)
(50, 50)


In [46]:
len_train_tags

49

In [47]:
df= pd.read_csv("test_small.csv")

In [48]:

test_data_dict = pd.Series(df.untagged_sentence.values,index=df.id).to_dict()

In [49]:

def viterbi_algorithm_new(sentence, transition_prob_matrix,emission_prob_matrix):
    prob_matrix= np.zeros((len_train_tags, len_train_tokens))
    m,n= prob_matrix.shape
    len_of_sentence=len(sentence)
    bp= np.zeros((m,n))
    tags=[]
    for i in range(len(sentence)):
        for j in range(m):
            
            if sentence[i] not in dict_train_tokens:
                emission_prob=1
            else:
                emission_prob=emission_prob_matrix[j][dict_tokens_to_index[sentence[i]]]
            if(i==0):
                prob_matrix[j][i]=transition_prob_matrix[len_train_tags][j]*emission_prob
            else:
                prev_col= prob_matrix[:,i-1].reshape(1,-1).copy()
                for k in range(m):
                    prev_col[0][k]*=(transition_prob_matrix[k][j]*emission_prob)
                maxpos=np.argmax(prev_col)
                prob_matrix[j][i]=prev_col[0][maxpos]
                bp[j][i]=maxpos
    last_tag_index=np.argmax(prob_matrix[:,-1])
    tags.append(last_tag_index)

    for i in range(len_of_sentence-1,0,-1):
        tags.insert(0,int(bp[tags[0],i]))
    tagged_sentence=[]
    for i in range(len_of_sentence):
        tagged_sentence.append((sentence[i],dict_index_to_tags[tags[i]]))
    return tagged_sentence

    

In [52]:
def get_predictions(test_data_dict):
    data_list=[]
    predictions={}
    ctr=0
    for id in test_data_dict:
        data_list.append((id,test_data_dict[id]))
        ctr+=1
    for data_row in data_list:
        # print(data_row[0])
        temp=ast.literal_eval(data_row[1])
        tagged_sentence=viterbi_algorithm_new(temp,transition_prob_matrix,emission_prob_matrix)
        
        predictions[data_row[0]]= tagged_sentence
    return predictions

In [53]:
test_predictions=get_predictions(test_data_dict)

In [54]:
predictions_df = pd.DataFrame(test_predictions,columns=['id', 'tagged_sentence'])
output_csv_path = 'predictions_hmm.csv'
predictions_df.to_csv(output_csv_path, index=False)

In [55]:



csv_file_path = 'predictions_hmm.csv'


with open(csv_file_path, 'w', newline='') as file:
    writer = csv.writer(file)

    writer.writerow(['id', 'tagged_sentence'])

    for key, values in test_predictions.items():
        writer.writerow([key, values])


csv_file_path


'predictions_hmm.csv'